In [2]:
import torch
import numpy as np

print(torch.__version__)
print(torch.backends.mps.is_available())

2.4.0
True


In [11]:
# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)
print('tensor0d:', tensor0d.dtype)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])
print('tensor1d:', tensor1d.dtype)
# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2], 
                         [3, 4]])
print('tensor2d:', tensor2d.dtype)
# create a 3D tensor from a nested Python list
tensor3d_1 = torch.tensor([[[1, 2], [3, 4]], 
                           [[5, 6], [7, 8]]])
print('tensor3d_1:', tensor3d_1.dtype)
# create a 3D tensor from NumPy array
ary3d = np.array([[[1, 2], [3, 4]], 
                  [[5, 6], [7, 8]]])  # Specify dtype as int32
tensor3d_2 = torch.tensor(ary3d)  # Copies NumPy array
print('tensor3d_2:', tensor3d_2.dtype)
tensor3d_3 = torch.from_numpy(ary3d)  # Shares memory with NumPy array
print('tensor3d_3:', tensor3d_3.dtype)

ary3d[0][0][0] = 100

floatvec = tensor1d.to(torch.float32)
print('floatvec:', floatvec.dtype)

# Print the tensors to verify
print()
print("tensor3d_2:", tensor3d_2)
print()
print("tensor3d_3:", tensor3d_3)

tensor0d: torch.int64
tensor1d: torch.int64
tensor2d: torch.int64
tensor3d_1: torch.int64
tensor3d_2: torch.int64
tensor3d_3: torch.int64
floatvec: torch.float32

tensor3d_2: tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

tensor3d_3: tensor([[[100,   2],
         [  3,   4]],

        [[  5,   6],
         [  7,   8]]])


In [13]:
import torch.nn.functional as F

y = torch.tensor([1.0])  # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0])  # bias unit

z = x1 * w1 + b          # net input
a = torch.sigmoid(z)     # activation & output

loss = F.binary_cross_entropy(a, y)
print(loss)

tensor(0.0852)


In [14]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b 
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [15]:
loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [16]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
                
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [17]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [18]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [19]:
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.1089, -0.0671,  0.0185,  ..., -0.1091,  0.0921,  0.0439],
        [-0.0212, -0.0438,  0.0105,  ..., -0.0768,  0.0261,  0.0271],
        [-0.0784, -0.1351,  0.1314,  ..., -0.1262, -0.0049, -0.0533],
        ...,
        [-0.0288, -0.0327,  0.0076,  ..., -0.0698, -0.0825,  0.0503],
        [-0.1041,  0.1085, -0.0696,  ..., -0.1131,  0.1017,  0.1196],
        [ 0.0384,  0.1066,  0.0136,  ...,  0.1385,  0.0803,  0.1304]],
       requires_grad=True)


In [20]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [21]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [22]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [23]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [24]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])
